<a href="https://colab.research.google.com/github/sk1210/object_detection_wheat/blob/master/wheat_head_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install wandb
!unzip /content/Dataset/global-wheat-detection.zip  -d /content/Dataset/
import glob
print (len(glob.glob("/content/Dataset/train/*.jpg"))) # 3422

Archive:  /content/Dataset/global-wheat-detection.zip
replace /content/Dataset/test/2fd875eaa.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
3422


In [35]:
!pip install wandb

     |████████████████████████████████| 1.4MB 3.3MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 163kB 17.5MB/s 
     |████████████████████████████████| 112kB 23.0MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=a98167776d58726f7ee455670babfba0c33964cc2a25982a05e4719de2aabcc5
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=b33e6b3fb564948fa029f3cea7f73518f16733de5e9e05ffc4dbe1473347fc47
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=b026

In [21]:
# connect to gdrive 
# install utils code

!git clone https://github.com/sk1210/object_detection_wheat.git
!mkdir /content/Dataset
!cp  "/content/drive/My Drive/Dataset/global-wheat-detection.zip" /content/Dataset/
!unzip /content/Dataset/global-wheat-detection.zip  /content/Dataset/
import os
os.chdir("/content/object_detection_wheat")


fatal: destination path 'object_detection_wheat' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/Dataset’: File exists
Archive:  /content/Dataset/global-wheat-detection.zip
caution: filename not matched:  /content/Dataset/


In [37]:
# data loader

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torch
import cv2
import csv
import os
from transforms import ToTensor
from torchvision.transforms import functional as F
import transforms, utils
import sys


class WheatData(Dataset):
    def __init__(self, csv_path="", mode = "train"):
        self.csv_path = r"/content/drive/My Drive/Dataset/train.csv"
        self.images_path = r"/content/Dataset/train/"
        self.mode = mode
        self.transforms = transforms.RandomHorizontalFlip(0.5)
        self.images_info = {}
        self.total_samples = None
        self.image_scale = 604
        self.scaleFactor = self.image_scale / 1024  # 0.25

        self.read_csv()


    def set_image_scale(self, image_scale):
        self.image_scale = image_scale
        self.scaleFactor = self.image_scale / 1024  # 0.25

    def write_images(self, out_dir):
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        for key, data in self.images_info.items():
            img_path = os.path.join(self.images_path, key + ".jpg")
            boxes = data["bbox"]

            img = cv2.imread(img_path)

            # draw boxes
            for box in boxes:
                x1, y1, w, h = list(map(int, box))
                cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 100, 0), 3)
                cv2.imwrite(os.path.join(out_dir, os.path.basename(img_path)), img)

    def read_csv(self):
        print("Loading CSV ")
        with open(self.csv_path) as f:
            reader = csv.reader(f)
            header = next(reader)
            for row in reader:
                image_id = row[0]
                if not (image_id in self.images_info):
                    self.images_info[image_id] = {}

                for key, value in zip(header, row):
                    if key == "bbox":
                        box = value.replace("[", "").replace("]", "").replace(" ", "").split(",")
                        x1, y1, w, h = list(map(float, box))
                        box = [x1, y1, x1 + w, y1 + h]
                        if key in self.images_info[image_id]:
                            self.images_info[image_id][key].append(box)
                        else:
                            self.images_info[image_id][key] = [box]
                    else:
                        self.images_info[image_id][key] = value
                # print (self.images_info)
        self.key_lists = list(self.images_info.keys())
        self.key_lists.sort()
        self.key_list = self.key_lists[:-100]
        self.key_list = self.key_lists
        self.total_samples = len(self.key_list)

    def apply_transorms(self, img, target):

        hor_flip = transforms.RandomHorizontalFlip(0.5)
        ver_flip = transforms.RandomVerticalFlip(0.4)
        jitter = transforms.ColorJitter(brightness=0.3, contrast=0.2, hue=0.1)

        if self.mode == "train":
            img = jitter(img)
        img = F.to_tensor(img)
        if self.mode == "train":
            img, target = hor_flip(img, target)
            img, target = ver_flip(img, target)

        return img, target

    def __getitem__(self, idx):

        key = self.key_list[idx]
        #print(idx, key)
        data = self.images_info[key]
        img_path = os.path.join(self.images_path, key + ".jpg")
        boxes = data["bbox"]

        img = Image.open(img_path).convert("RGB")
        width, height = int(img.width * self.scaleFactor), int(img.height * self.scaleFactor)
        img = img.resize((width, height))
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32) * self.scaleFactor
        labels = torch.ones((len(boxes),), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
        if self.mode == "test":
            target["image_id"] = torch.tensor([idx])
            target["area"] = area
            target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.apply_transorms(img, target)

        return img, target

    def __len__(self):
        return self.total_samples


if __name__ == "__main__":
    dataset = WheatData()
    print("END")
    # image_data.write_images("kaggle/temp/output")

    dataloader = DataLoader(dataset, batch_size=10, shuffle=True, collate_fn=utils.collate_fn)
    #print(next(iter(dataloader)))


Loading CSV 
END


In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch
from torchvision.models.detection.backbone_utils import IntermediateLayerGetter
from collections import OrderedDict
from torch import nn
from torchvision.ops import misc as misc_nn_ops
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork, LastLevelMaxPool
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models import resnext50_32x4d, resnet101, resnext101_32x8d
from torchvision.models.detection.rpn import AnchorGenerator
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}

class BackboneWithFPN(nn.Sequential):
    """
    Adds a FPN on top of a model.

    Internally, it uses torchvision.models._utils.IntermediateLayerGetter to
    extract a submodel that returns the feature maps specified in return_layers.
    The same limitations of IntermediatLayerGetter apply here.

    Arguments:
        backbone (nn.Module)
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
        in_channels_list (List[int]): number of channels for each feature map
            that is returned, in the order they are present in the OrderedDict
        out_channels (int): number of channels in the FPN.

    Attributes:
        out_channels (int): the number of channels in the FPN
    """
    def __init__(self, backbone, return_layers, in_channels_list, out_channels):
        body = IntermediateLayerGetter(backbone, return_layers=return_layers)
        fpn = FeaturePyramidNetwork(
            in_channels_list=in_channels_list,
            out_channels=out_channels,
            extra_blocks=LastLevelMaxPool(),
        )
        super(BackboneWithFPN, self).__init__(OrderedDict(
            [("body", body), ("fpn", fpn)]))
        self.out_channels = out_channels


def resnet_fpn_backbone(backbone_name, pretrained):
    #backbone = resnet.__dict__[backbone_name](
    # backbone = resnext101_32x8d(
    #     pretrained=pretrained,
    #     norm_layer=misc_nn_ops.FrozenBatchNorm2d)
    backbone = torch.hub.load('pytorch/vision:v0.6.0', backbone_name, pretrained=True)
    # freeze layers
    for name, parameter in backbone.named_parameters():
        if 'layer2' not in name and 'layer3' not in name and 'layer4' not in name:
            parameter.requires_grad_(False)

    return_layers = {'layer1': 0, 'layer2': 1, 'layer3': 2, 'layer4': 3}

    in_channels_stage2 = 256
    in_channels_list = [
        in_channels_stage2,
        in_channels_stage2 * 2,
        in_channels_stage2 * 4,
        in_channels_stage2 * 8,
    ]
    out_channels = 256
    return BackboneWithFPN(backbone, return_layers, in_channels_list, out_channels)

def _fasterrcnn_resnext100(pretrained=False, progress=True,
                            num_classes=91, pretrained_backbone=True, **kwargs):

    backbone = resnet_fpn_backbone('resnext101_32x8d', pretrained_backbone)
    model = FasterRCNN(backbone, num_classes, **kwargs)

    return model


def faster_rcnn_resnext(num_classes):
    # load a model pre-trained pre-trained on COCO

    model = _fasterrcnn_resnext100(pretrained=False, pretrained_backbone = True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

if __name__ == "__main__":
    model = faster_rcnn_resnext(2)
    print(model)

In [40]:
import torch
# from data_loader import WheatData as Dataset
#from model import faster_rcnn, faster_rcnn_101, faster_rcnn_custom
#from model_custom import faster_rcnn_resnext
import utils
from engine import train_one_epoch
import random

weight_dir = r"/content/drive/My Drive/Dataset/Wheat_head_detection/weights/resnext101/"

weight_file = r"kaggle\weights\custom_anchors\24.weight"
start_epoch = 0

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = WheatData()
    # dataset_test = Dataset()

    # split the dataset in train and test set
    indices = list(range(len(dataset))) #.randperm(len(dataset)).tolist()
    # dataset = torch.utils.data.Subset(dataset, indices[:-100])
    # dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1,
        collate_fn=utils.collate_fn)

    # data_loader_test = torch.utils.data.DataLoader(
    #     dataset_test, batch_size=1, shuffle=False, num_workers=4,
    #     collate_fn=utils.collate_fn)

    # get the model using our helper function
    # model = faster_rcnn(num_classes)
    # model = faster_rcnn_101(num_classes)
    # model = faster_rcnn_custom(num_classes)
    model = faster_rcnn_resnext(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=7,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 35
    print("start training")
    #wandb.watch(model)
    image_scales = [512, 512, 604, 604, 604]
  
    #model.load_state_dict(torch.load(weight_file))

    for epoch in range(start_epoch, num_epochs):
        # train for one epoch, printing every 10 iterations
        image_scale = random.choice(image_scales)
        dataset.set_image_scale(image_scale)
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10, image_scale = image_scale)
        # update the learning rate
        torch.save(model.state_dict(), weight_dir+str(epoch) + ".weight")
        lr_scheduler.step()

        # evaluate on the test data set
        # evaluate(model, data_loader_test, device=device)

    print("That's it!")


if __name__ == '__main__':
    main()


Loading CSV 


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


start training


TypeError: ignored